<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9C%A0%EC%98%88%EC%8A%AC_%EB%8B%B5%EB%B3%80_%EC%83%9D%EC%84%B1_AI__%ED%82%A4%EC%9B%8C%EB%93%9C_%EC%B6%94%EC%B6%9C_%ED%95%99%EC%8A%B5_(%EC%A7%84%ED%96%89%EC%A4%91).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 특성 추출 사전학습 모델

- skt/kobert-base-v1 (131k download)
- BM-K/KoSimCSE-roberta (87k download)


In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install transformers

In [48]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# DATA_PATH = "/content/drive/MyDrive/생성 AI 모델링/data/"
DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device




'cpu'

# 데이터 정제

In [6]:
yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi_all.csv")


In [7]:
google_food = pd.read_csv(f"{DATA_PATH}playstore_food_reply_0904.csv")

## 정규표현식

In [8]:
train= yogiyo[['고객리뷰','사장댓글']]

rename_cols = ['review', 'reply']
train.columns = rename_cols
train.head()

,review,reply
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [9]:
review_list = []
for review in train['review']:
    if isinstance(review, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

<ipython-input-9-6374a3c305ac>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'] = review_list


,review,reply
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [13]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

<ipython-input-13-2764c488e75c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'] = reply_list


,review,reply
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...


# 리뷰 키워드 추출 해보기

## A. Keybert로 키워드 추출

In [15]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.8 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23765 sha256=547903ad3d0bc115ecb5dfb0f6c3323314c12b1ea567e6bd82896db340b5fa5e
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=7ee08bc9abc68690321468ea93ce8c9b72c46002f1d365b29981c379b0e66429
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers


### 1) skt/kobert-base-v1

In [35]:
from keybert import KeyBERT
# from transformers import XLNetTokenizer, BertTokenizer, BertModel
from tqdm import tqdm  # tqdm 추가
from transformers import AutoModel, AutoTokenizer


# keybert에 사용할 Kobert 모델 로드
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
model = AutoModel.from_pretrained("skt/kobert-base-v1")

# KeyBERT 모델 초기화
keybert_model = KeyBERT(model=model)

# 리뷰 데이터 (train['review']로 가정)
reviews = train['review'][:50]  # train['review']에 리뷰 데이터를 넣으세요
keywords_list = []

# tqdm을 사용하여 진행 상황 표시
for text in tqdm(reviews):
    keywords = keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
    keywords_list.append(keywords)

# # 결과 출력
# for idx, keywords in enumerate(keywords_list):
#     print(f"Review {idx + 1} Keywords:", keywords)

result_df = pd.DataFrame({'Review': reviews, 'Keywords': keywords_list})
result_df




100%|██████████| 50/50 [00:08<00:00,  6.07it/s]


,Review,Keywords
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"[(배달시간도, 0.7201), (많아요, 0.5974), (맞춰서, 0.5803)..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,"[(먹었습니다, 0.6059), (맛있게, 0.5016), (리뷰가, 0.3841)..."
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,"[(펩시콜라는, 0.4169), (먹었습니다, 0.2331), (못했습니다, 0.2..."
3,굿 말이 필요 없는 메뉴 였습니다,"[(말이, 0.6761), (필요, 0.6469), (메뉴, 0.6346), (없는..."
4,맛있어요 양도많구좋음,"[(양도많구좋음, 0.9856), (맛있어요, 0.4829)]"
5,양많고 맛있었어요,"[(양많고, 0.9022), (맛있었어요, 0.6518)]"
6,처음 주문했는데 맛있어서 다먹었어요 리뷰 묵사발도 꼭 드세요! 맛있게 잘먹었습니다 ...,"[(맛있어서, 0.8302), (잘먹었습니다, 0.7657), (다먹었어요, 0.7..."
7,가성비 아주 만족 합니다. 맛있게 잘먹었습니다.,"[(만족, 0.4216), (잘먹었습니다, 0.4133), (맛있게, 0.4087)..."
8,맛있게 먹었습니다!,"[(먹었습니다, 0.6755), (맛있게, 0.603)]"
9,새벽부터 시켰는데 배송 시간 맞춰서 잘 와주셨고 도시락도 맛있고 묵사발도 맛있습니다~!,"[(맛있습니다, 0.7125), (맛있고, 0.6453), (새벽부터, 0.5273..."


### 2) BM-K/KoSimCSE-roberta

In [30]:
import pandas as pd
from keybert import KeyBERT
from tqdm import tqdm
# from transformers import BertTokenizer, RobertaTokenizer, RobertaModel
from transformers import AutoModel, AutoTokenizer



# BM-K/KoSimCSE-roberta 모델 로드
tokenizer = AutoTokenizer.from_pretrained("BM-K/KoSimCSE-roberta")
model = AutoModel.from_pretrained("BM-K/KoSimCSE-roberta")

# KeyBERT 모델 초기화
keybert_model = KeyBERT(model=model)

# 리뷰 데이터 (train['review']로 가정)
reviews = train['review'][:50]  # train['review']에 리뷰 데이터를 넣으세요
keywords_list = []

# tqdm을 사용하여 진행 상황 표시
for text in tqdm(reviews):
    keywords = keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=5)
    keywords_list.append(keywords)

# 데이터 프레임으로 변환
result_df = pd.DataFrame({'Review': reviews, 'Keywords': keywords_list})
result_df


100%|██████████| 50/50 [00:15<00:00,  3.21it/s]


,Review,Keywords
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"[(배달시간도, 0.7201), (많아요, 0.5974), (맞춰서, 0.5803)..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,"[(먹었습니다, 0.6059), (맛있게, 0.5016), (리뷰가, 0.3841)..."
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,"[(펩시콜라는, 0.4169), (먹었습니다, 0.2331), (못했습니다, 0.2..."
3,굿 말이 필요 없는 메뉴 였습니다,"[(말이, 0.6761), (필요, 0.6469), (메뉴, 0.6346), (없는..."
4,맛있어요 양도많구좋음,"[(양도많구좋음, 0.9856), (맛있어요, 0.4829)]"
5,양많고 맛있었어요,"[(양많고, 0.9022), (맛있었어요, 0.6518)]"
6,처음 주문했는데 맛있어서 다먹었어요 리뷰 묵사발도 꼭 드세요! 맛있게 잘먹었습니다 ...,"[(맛있어서, 0.8302), (잘먹었습니다, 0.7657), (다먹었어요, 0.7..."
7,가성비 아주 만족 합니다. 맛있게 잘먹었습니다.,"[(만족, 0.4216), (잘먹었습니다, 0.4133), (맛있게, 0.4087)..."
8,맛있게 먹었습니다!,"[(먹었습니다, 0.6755), (맛있게, 0.603)]"
9,새벽부터 시켰는데 배송 시간 맞춰서 잘 와주셨고 도시락도 맛있고 묵사발도 맛있습니다~!,"[(맛있습니다, 0.7125), (맛있고, 0.6453), (새벽부터, 0.5273..."


### 3) distilbert-base-nli-mean-tokens
- 이 모델이 Keybert의 디폴트 모델인거 같음

In [32]:
from keybert import KeyBERT
from tqdm import tqdm


# KeyBERT 모델 초기화 (distilbert-base-nli-mean-tokens 사용)
keybert_model = KeyBERT('distilbert-base-nli-mean-tokens')

# 리뷰 데이터 (train['review']로 가정)
reviews = train['review'][:50]  # train['review']에 리뷰 데이터를 넣으세요
keywords_list = []

# tqdm을 사용하여 진행 상황 표시
for text in tqdm(reviews):
    keywords = keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=5)
    keywords_list.append(keywords)

# 결과 출력
result_df = pd.DataFrame({'Review': reviews, 'Keywords': keywords_list})
result_df


100%|██████████| 50/50 [00:29<00:00,  1.70it/s]


,Review,Keywords
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"[(배달시간도, 0.9154), (사진보다, 0.8853), (실물이, 0.7395..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,"[(리뷰가, 0.8918), (시킬게요, 0.8882), (먹었습니다, 0.3528..."
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,"[(펩시콜라는, 0.9163), (주문에, 0.847), (다만, 0.806), (..."
3,굿 말이 필요 없는 메뉴 였습니다,"[(말이, 0.8651), (필요, 0.8422), (메뉴, 0.822), (였습니..."
4,맛있어요 양도많구좋음,"[(양도많구좋음, 0.9864), (맛있어요, 0.9864)]"
5,양많고 맛있었어요,"[(양많고, 0.9864), (맛있었어요, 0.9864)]"
6,처음 주문했는데 맛있어서 다먹었어요 리뷰 묵사발도 꼭 드세요! 맛있게 잘먹었습니다 ...,"[(주문할게요, 0.8756), (묵사발도, 0.8605), (다음에, 0.7495..."
7,가성비 아주 만족 합니다. 맛있게 잘먹었습니다.,"[(가성비, 0.9011), (합니다, 0.8724), (만족, 0.8706), (..."
8,맛있게 먹었습니다!,"[(먹었습니다, 0.951), (맛있게, 0.951)]"
9,새벽부터 시켰는데 배송 시간 맞춰서 잘 와주셨고 도시락도 맛있고 묵사발도 맛있습니다~!,"[(묵사발도, 0.8754), (새벽부터, 0.8632), (도시락도, 0.8031..."


### 4) monologg/kobert

In [34]:
import pandas as pd
from keybert import KeyBERT
from tqdm import tqdm
# from transformers import BertTokenizer, RobertaTokenizer, RobertaModel
from transformers import AutoModel, AutoTokenizer



# BM-K/KoSimCSE-roberta 모델 로드
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model = AutoModel.from_pretrained("monologg/kobert")

# KeyBERT 모델 초기화
keybert_model = KeyBERT(model=model)

# 리뷰 데이터 (train['review']로 가정)
reviews = train['review'][:50]  # train['review']에 리뷰 데이터를 넣으세요
keywords_list = []

# tqdm을 사용하여 진행 상황 표시
for text in tqdm(reviews):
    keywords = keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
    keywords_list.append(keywords)

# 데이터 프레임으로 변환
result_df = pd.DataFrame({'Review': reviews, 'Keywords': keywords_list})
result_df



100%|██████████| 50/50 [00:07<00:00,  7.02it/s]


,Review,Keywords
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"[(배달시간도, 0.7201), (많아요, 0.5974), (맞춰서, 0.5803)..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,"[(먹었습니다, 0.6059), (맛있게, 0.5016), (리뷰가, 0.3841)..."
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,"[(펩시콜라는, 0.4169), (먹었습니다, 0.2331), (못했습니다, 0.2..."
3,굿 말이 필요 없는 메뉴 였습니다,"[(말이, 0.6761), (필요, 0.6469), (메뉴, 0.6346), (없는..."
4,맛있어요 양도많구좋음,"[(양도많구좋음, 0.9856), (맛있어요, 0.4829)]"
5,양많고 맛있었어요,"[(양많고, 0.9022), (맛있었어요, 0.6518)]"
6,처음 주문했는데 맛있어서 다먹었어요 리뷰 묵사발도 꼭 드세요! 맛있게 잘먹었습니다 ...,"[(맛있어서, 0.8302), (잘먹었습니다, 0.7657), (다먹었어요, 0.7..."
7,가성비 아주 만족 합니다. 맛있게 잘먹었습니다.,"[(만족, 0.4216), (잘먹었습니다, 0.4133), (맛있게, 0.4087)..."
8,맛있게 먹었습니다!,"[(먹었습니다, 0.6755), (맛있게, 0.603)]"
9,새벽부터 시켰는데 배송 시간 맞춰서 잘 와주셨고 도시락도 맛있고 묵사발도 맛있습니다~!,"[(맛있습니다, 0.7125), (맛있고, 0.6453), (새벽부터, 0.5273..."


## B. 키위로 키워드 추출

# skt Kobert 이용, 학습 및 답변 확인

- 긍부정지수 추가

In [50]:
train_posneg = pd.read_csv("train_posneg.csv")

In [51]:
cols = ['review','Unnamed: 0']
train_posneg = train_posneg.drop(columns=cols)
train_posneg

,sentiment,score
0,긍정,0.667125
1,긍정,0.751605
2,긍정,0.445775
3,긍정,0.741268
4,긍정,0.590259
...,...,...
18681,긍정,0.491652
18682,긍정,0.557971
18683,긍정,0.487945
18684,긍정,0.512616


In [52]:
train = pd.concat([train,train_posneg],axis=1,ignore_index=True)
train.columns = ['review','reply','sentiment','score']

- 맞춤법 처리

In [53]:
train

,review,reply,sentiment,score
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...",긍정,0.667125
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,긍정,0.751605
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,긍정,0.445775
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,긍정,0.741268
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,긍정,0.590259
...,...,...,...,...
18681,맛있게 잘 먹었어요,감사합니다! 다음에도 이용해주세용,긍정,0.491652
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 ! 다음에도 애용해주세용,긍정,0.557971
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다! 다음에도 이용해주세용,긍정,0.487945
18684,잘 먹었어요~,감사합니다! 다음에도 이용해주세용,긍정,0.512616


- 임시 3000개 추출

In [56]:
shuffled_train = train.sample(frac=1)[:3000]
shuffled_train

,review,reply,sentiment,score
16908,덮밥은 양도많고 맛있어요 근데 치즈돈까스가 기름을 하두먹어서 못먹을정도로 너무 느끼...,소중한 리뷰작성 진심으로 감사드립니다 저희 진지덮밥을 찾아주셔서 너무나 감사합니다 ...,부정,0.667395
14633,맛있게 잘 먹었습니다,맛있게 드셨다니 감사합니다 연휴 잘 보내시고 무더위 건강 유의 하세요,긍정,0.538841
14876,좀 싸다고 생각할수잇는 밥이지만 소홀히안하고 관리하시는거 보면 여기 사장님이 고객대...,이쁜 리뷰 감사합니다 고객님 맛있게 드셨다니 너무 감사합니다 정성스러운 리뷰와 너무...,긍정,0.904239
3613,정말 너무 맛있어요 최고,만족하시는 식사가 되신 것 같아서 정말 기쁩니다 고객님들이 맛있게 식사하시고 정성스...,긍정,0.896239
16290,세트 시켰을때 탕수육 너무 맛있어서 이번엔 탕수육 시켜봤는데 이제 여기서만 먹을게요...,"안녕하세요 항상 부족함 없는 식사 하실수 있도록 초심 잃지않고 좋은재료, 정성담은 ...",긍정,0.858516
...,...,...,...,...
14682,왠만하면 리뷰안쓰는데 맛있게 잘먹었습니다~ 사장님~ 번창하세요,정말 왠만하면 리뷰 보기 힘든 요기요인데 너무 감사합니다 요즘 날씨가 벌써 덥네요 ...,긍정,0.822442
9233,매운맛은 맵기는 한데 정말 맛이 없네요 그리고 콜라는 맛이 이상해서 유통기한까지 확...,에고에고.그래도 아직은 부족한 게 많은 짱닭이네요 이번 주문에 매운맛이 고객님께는 ...,부정,0.748549
7937,쫄면은 자성당이 최애맛집,안녕하세요 co 님 소중한 리뷰 고맙습니다 자주 찾아주시니 그또한 감사합니다 변치않...,긍정,0.442815
12368,매번 먹어도 질리지 않고 맛도 좋아요~,be 님 소중한 주문과 좋은 리뷰말씀 진심으로 감사 드립니다. 고객님께서 언제 어디...,긍정,0.646608


- 리뷰에 대해 키워드만 추출

In [59]:
from keybert import KeyBERT
# from transformers import XLNetTokenizer, BertTokenizer, BertModel
from tqdm import tqdm  # tqdm 추가
from transformers import AutoModel, AutoTokenizer


# keybert에 사용할 Kobert 모델 로드
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
model = AutoModel.from_pretrained("skt/kobert-base-v1")

# KeyBERT 모델 초기화
keybert_model = KeyBERT(model=model)

# 리뷰 데이터 (train['review']로 가정)
reviews = shuffled_train['review'] # 셔플로 섞은 3000개에 대해서 키워드 추출함
keywords_list = []

# tqdm을 사용하여 진행 상황 표시
for text in tqdm(reviews):
    keywords = keybert_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
    keywords = [keyword[0] for keyword in keywords]
    keywords_list.append(keywords)

# # 결과 출력
# for idx, keywords in enumerate(keywords_list):
#     print(f"Review {idx + 1} Keywords:", keywords)

result_df = pd.DataFrame({'Keywords': keywords_list})
result_df


100%|██████████| 3000/3000 [09:51<00:00,  5.07it/s]


,Keywords
0,"[맛있어요, 기름을, 치즈돈까스가, 너무, 덮밥은, 못먹을정도로, 하두먹어서, 느끼..."
1,"[맛있게, 먹었습니다]"
2,"[사장님이, 밥이지만, 고객대하는, 관리하시는거, 생각할수잇는, 다음에, 되네여, ..."
3,"[맛있어요, 최고, 정말, 너무]"
4,"[맛있어서, 존맛입니다, 탕수육, 먹을게요, 시켰을때, 시켜봤는데, 이제, 여기서만..."
...,...
2995,"[번창하세요, 잘먹었습니다, 맛있게, 왠만하면, 사장님, 리뷰안쓰는데]"
2996,"[전화도, 전화, 콜라는, 없네요, 안받으시고, 어떻게하신건지, 상한거, 같아요, ..."
2997,"[쫄면은, 최애맛집, 자성당이]"
2998,"[먹어도, 맛도, 좋아요, 않고, 매번, 질리지]"


In [69]:
result_df = result_df.reset_index()[['Keywords']]
result_df

,Keywords
0,"[맛있어요, 기름을, 치즈돈까스가, 너무, 덮밥은, 못먹을정도로, 하두먹어서, 느끼..."
1,"[맛있게, 먹었습니다]"
2,"[사장님이, 밥이지만, 고객대하는, 관리하시는거, 생각할수잇는, 다음에, 되네여, ..."
3,"[맛있어요, 최고, 정말, 너무]"
4,"[맛있어서, 존맛입니다, 탕수육, 먹을게요, 시켰을때, 시켜봤는데, 이제, 여기서만..."
...,...
2995,"[번창하세요, 잘먹었습니다, 맛있게, 왠만하면, 사장님, 리뷰안쓰는데]"
2996,"[전화도, 전화, 콜라는, 없네요, 안받으시고, 어떻게하신건지, 상한거, 같아요, ..."
2997,"[쫄면은, 최애맛집, 자성당이]"
2998,"[먹어도, 맛도, 좋아요, 않고, 매번, 질리지]"


In [70]:
train = shuffled_train.reset_index()[['review','reply','sentiment','score']]
train

,review,reply,sentiment,score
0,덮밥은 양도많고 맛있어요 근데 치즈돈까스가 기름을 하두먹어서 못먹을정도로 너무 느끼...,소중한 리뷰작성 진심으로 감사드립니다 저희 진지덮밥을 찾아주셔서 너무나 감사합니다 ...,부정,0.667395
1,맛있게 잘 먹었습니다,맛있게 드셨다니 감사합니다 연휴 잘 보내시고 무더위 건강 유의 하세요,긍정,0.538841
2,좀 싸다고 생각할수잇는 밥이지만 소홀히안하고 관리하시는거 보면 여기 사장님이 고객대...,이쁜 리뷰 감사합니다 고객님 맛있게 드셨다니 너무 감사합니다 정성스러운 리뷰와 너무...,긍정,0.904239
3,정말 너무 맛있어요 최고,만족하시는 식사가 되신 것 같아서 정말 기쁩니다 고객님들이 맛있게 식사하시고 정성스...,긍정,0.896239
4,세트 시켰을때 탕수육 너무 맛있어서 이번엔 탕수육 시켜봤는데 이제 여기서만 먹을게요...,"안녕하세요 항상 부족함 없는 식사 하실수 있도록 초심 잃지않고 좋은재료, 정성담은 ...",긍정,0.858516
...,...,...,...,...
2995,왠만하면 리뷰안쓰는데 맛있게 잘먹었습니다~ 사장님~ 번창하세요,정말 왠만하면 리뷰 보기 힘든 요기요인데 너무 감사합니다 요즘 날씨가 벌써 덥네요 ...,긍정,0.822442
2996,매운맛은 맵기는 한데 정말 맛이 없네요 그리고 콜라는 맛이 이상해서 유통기한까지 확...,에고에고.그래도 아직은 부족한 게 많은 짱닭이네요 이번 주문에 매운맛이 고객님께는 ...,부정,0.748549
2997,쫄면은 자성당이 최애맛집,안녕하세요 co 님 소중한 리뷰 고맙습니다 자주 찾아주시니 그또한 감사합니다 변치않...,긍정,0.442815
2998,매번 먹어도 질리지 않고 맛도 좋아요~,be 님 소중한 주문과 좋은 리뷰말씀 진심으로 감사 드립니다. 고객님께서 언제 어디...,긍정,0.646608


In [73]:
tmp = pd.concat([train, result_df],axis=1)
tmp

,review,reply,sentiment,score,Keywords
0,덮밥은 양도많고 맛있어요 근데 치즈돈까스가 기름을 하두먹어서 못먹을정도로 너무 느끼...,소중한 리뷰작성 진심으로 감사드립니다 저희 진지덮밥을 찾아주셔서 너무나 감사합니다 ...,부정,0.667395,"[맛있어요, 기름을, 치즈돈까스가, 너무, 덮밥은, 못먹을정도로, 하두먹어서, 느끼..."
1,맛있게 잘 먹었습니다,맛있게 드셨다니 감사합니다 연휴 잘 보내시고 무더위 건강 유의 하세요,긍정,0.538841,"[맛있게, 먹었습니다]"
2,좀 싸다고 생각할수잇는 밥이지만 소홀히안하고 관리하시는거 보면 여기 사장님이 고객대...,이쁜 리뷰 감사합니다 고객님 맛있게 드셨다니 너무 감사합니다 정성스러운 리뷰와 너무...,긍정,0.904239,"[사장님이, 밥이지만, 고객대하는, 관리하시는거, 생각할수잇는, 다음에, 되네여, ..."
3,정말 너무 맛있어요 최고,만족하시는 식사가 되신 것 같아서 정말 기쁩니다 고객님들이 맛있게 식사하시고 정성스...,긍정,0.896239,"[맛있어요, 최고, 정말, 너무]"
4,세트 시켰을때 탕수육 너무 맛있어서 이번엔 탕수육 시켜봤는데 이제 여기서만 먹을게요...,"안녕하세요 항상 부족함 없는 식사 하실수 있도록 초심 잃지않고 좋은재료, 정성담은 ...",긍정,0.858516,"[맛있어서, 존맛입니다, 탕수육, 먹을게요, 시켰을때, 시켜봤는데, 이제, 여기서만..."
...,...,...,...,...,...
2995,왠만하면 리뷰안쓰는데 맛있게 잘먹었습니다~ 사장님~ 번창하세요,정말 왠만하면 리뷰 보기 힘든 요기요인데 너무 감사합니다 요즘 날씨가 벌써 덥네요 ...,긍정,0.822442,"[번창하세요, 잘먹었습니다, 맛있게, 왠만하면, 사장님, 리뷰안쓰는데]"
2996,매운맛은 맵기는 한데 정말 맛이 없네요 그리고 콜라는 맛이 이상해서 유통기한까지 확...,에고에고.그래도 아직은 부족한 게 많은 짱닭이네요 이번 주문에 매운맛이 고객님께는 ...,부정,0.748549,"[전화도, 전화, 콜라는, 없네요, 안받으시고, 어떻게하신건지, 상한거, 같아요, ..."
2997,쫄면은 자성당이 최애맛집,안녕하세요 co 님 소중한 리뷰 고맙습니다 자주 찾아주시니 그또한 감사합니다 변치않...,긍정,0.442815,"[쫄면은, 최애맛집, 자성당이]"
2998,매번 먹어도 질리지 않고 맛도 좋아요~,be 님 소중한 주문과 좋은 리뷰말씀 진심으로 감사 드립니다. 고객님께서 언제 어디...,긍정,0.646608,"[먹어도, 맛도, 좋아요, 않고, 매번, 질리지]"


- Keyword shuffled train 3000

In [74]:
train = tmp[['Keywords','reply','sentiment','score']]
train.columns = ['review','reply','sentiment','score']
train

train.to_csv(f'{DATA_PATH}keyword_shuffled_train_3000_0904.csv')

,review,reply,sentiment,score
0,"[맛있어요, 기름을, 치즈돈까스가, 너무, 덮밥은, 못먹을정도로, 하두먹어서, 느끼...",소중한 리뷰작성 진심으로 감사드립니다 저희 진지덮밥을 찾아주셔서 너무나 감사합니다 ...,부정,0.667395
1,"[맛있게, 먹었습니다]",맛있게 드셨다니 감사합니다 연휴 잘 보내시고 무더위 건강 유의 하세요,긍정,0.538841
2,"[사장님이, 밥이지만, 고객대하는, 관리하시는거, 생각할수잇는, 다음에, 되네여, ...",이쁜 리뷰 감사합니다 고객님 맛있게 드셨다니 너무 감사합니다 정성스러운 리뷰와 너무...,긍정,0.904239
3,"[맛있어요, 최고, 정말, 너무]",만족하시는 식사가 되신 것 같아서 정말 기쁩니다 고객님들이 맛있게 식사하시고 정성스...,긍정,0.896239
4,"[맛있어서, 존맛입니다, 탕수육, 먹을게요, 시켰을때, 시켜봤는데, 이제, 여기서만...","안녕하세요 항상 부족함 없는 식사 하실수 있도록 초심 잃지않고 좋은재료, 정성담은 ...",긍정,0.858516
...,...,...,...,...
2995,"[번창하세요, 잘먹었습니다, 맛있게, 왠만하면, 사장님, 리뷰안쓰는데]",정말 왠만하면 리뷰 보기 힘든 요기요인데 너무 감사합니다 요즘 날씨가 벌써 덥네요 ...,긍정,0.822442
2996,"[전화도, 전화, 콜라는, 없네요, 안받으시고, 어떻게하신건지, 상한거, 같아요, ...",에고에고.그래도 아직은 부족한 게 많은 짱닭이네요 이번 주문에 매운맛이 고객님께는 ...,부정,0.748549
2997,"[쫄면은, 최애맛집, 자성당이]",안녕하세요 co 님 소중한 리뷰 고맙습니다 자주 찾아주시니 그또한 감사합니다 변치않...,긍정,0.442815
2998,"[먹어도, 맛도, 좋아요, 않고, 매번, 질리지]",be 님 소중한 주문과 좋은 리뷰말씀 진심으로 감사 드립니다. 고객님께서 언제 어디...,긍정,0.646608


In [75]:
train_original = tmp[['review','reply','sentiment','score']]
train_original

train_original.to_csv(f'{DATA_PATH}original_shuffled_train_3000_0904.csv')

# 전처리

In [76]:
train

,review,reply,sentiment,score
0,"[맛있어요, 기름을, 치즈돈까스가, 너무, 덮밥은, 못먹을정도로, 하두먹어서, 느끼...",소중한 리뷰작성 진심으로 감사드립니다 저희 진지덮밥을 찾아주셔서 너무나 감사합니다 ...,부정,0.667395
1,"[맛있게, 먹었습니다]",맛있게 드셨다니 감사합니다 연휴 잘 보내시고 무더위 건강 유의 하세요,긍정,0.538841
2,"[사장님이, 밥이지만, 고객대하는, 관리하시는거, 생각할수잇는, 다음에, 되네여, ...",이쁜 리뷰 감사합니다 고객님 맛있게 드셨다니 너무 감사합니다 정성스러운 리뷰와 너무...,긍정,0.904239
3,"[맛있어요, 최고, 정말, 너무]",만족하시는 식사가 되신 것 같아서 정말 기쁩니다 고객님들이 맛있게 식사하시고 정성스...,긍정,0.896239
4,"[맛있어서, 존맛입니다, 탕수육, 먹을게요, 시켰을때, 시켜봤는데, 이제, 여기서만...","안녕하세요 항상 부족함 없는 식사 하실수 있도록 초심 잃지않고 좋은재료, 정성담은 ...",긍정,0.858516
...,...,...,...,...
2995,"[번창하세요, 잘먹었습니다, 맛있게, 왠만하면, 사장님, 리뷰안쓰는데]",정말 왠만하면 리뷰 보기 힘든 요기요인데 너무 감사합니다 요즘 날씨가 벌써 덥네요 ...,긍정,0.822442
2996,"[전화도, 전화, 콜라는, 없네요, 안받으시고, 어떻게하신건지, 상한거, 같아요, ...",에고에고.그래도 아직은 부족한 게 많은 짱닭이네요 이번 주문에 매운맛이 고객님께는 ...,부정,0.748549
2997,"[쫄면은, 최애맛집, 자성당이]",안녕하세요 co 님 소중한 리뷰 고맙습니다 자주 찾아주시니 그또한 감사합니다 변치않...,긍정,0.442815
2998,"[먹어도, 맛도, 좋아요, 않고, 매번, 질리지]",be 님 소중한 주문과 좋은 리뷰말씀 진심으로 감사 드립니다. 고객님께서 언제 어디...,긍정,0.646608


- 고객 id 처리

In [88]:
train = pd.DataFrame(train)
train['reply'] = train['reply'].str.replace(r'\w+\s*님', '고객님', regex=True)


- 가게명 처리

In [78]:
ygy_store_lst = ["1988응답하라추억의옛날도시락-방이점", "워커스커피 로스터스", "떡깨비-가락점", "당치땡-헬리오시티점", "우시장국밥-석촌점",  "고돼지-송파점",  "마라의신마라탕&마라샹궈",  "춘리마라탕-송파점",  "손수경의육회담은연어-가락점",  "무국적식탁-송파점",  "청년치킨-삼전점",  "송파만다린",  "배달의쌀국수-송파점",  "닭강점-헬리오시티점",  "토핑폭탄김치찜&김치찌개-송파점",  "조형훈족발보쌈",  "여왕의브런치",  "모어댄버터",  "플러스82-송파점",  "더티베지",  "잭아저씨족발보쌈-본점",  "육회&연어-미아본점",  "갈비민족-본점",  "구구족-성신여대역점",  "피자보이시나-대학로점",  "왕빈자삼파전-미아본점",  "행복한찜닭-성북점",  "스시사소우",  "저팔계&중국&마라탕&덮밥&요리&전문점",  "명작파스타-성북점",  "올데이파스타-동대문구점",  "대한냉면-성북점",  "인생제육",  "고기듬뿍미트박스-성북점",  "피자스웨그-성북점",  "청춘식당묵은지김치찜",  "삽교원조두리곱창",  "뜸들이다-북서울꿈의숲점",  "더팔당매운오징어&닭발 김치찜",  "우리집반찬도시락-혜화막내딸지점",  "수피자",  "마라쿡",  "김포불닭발",  "헬로팬닭갈비&마약볶음밥",  "대찌 대파생고기김치찌개&정육왕국물두루치기",  "구래상회",  "승도리네곱도리탕",  "자성당",  "수락",  "나인곱창",  "어시장",  "돈까스회관",  "호랑이초밥",  "땡초곱창막창",  "피자이탈리",  "텐텐마라탕",  "과일에반하다",  "삼다수",  "병천토속순대",  "최고남제육명가",  "짱닭치킨",  "걸작떡볶이",  "직화삼겹직구삼",  "짜글이가돼었소",  "부대찌개자신있는집",  "토핑폭탄김치찜&김치찌개-부천점",  "1989마라탕-부천점",  "미친피자-부평점",  "천년닭강정-부천부평점",  "수상한삼겹살-부천점",  "수제죽전문점-본가진죽-본점",  "오늘은 분식",  "이끌림마라탕",  "탐나는피자-부평점",  "고기혁명-부천점",  "라화방마라탕-신중동점",  "전금례닭볶음탕-부천점",  "큰아들백순대",  "공주닭발-부천시청점",  "신자매김치찜&김치찌개",  "짱닭치킨-도촌점",  "이태리면가게-야탑점",  "폴트버거-판교점",  "랜돌프뉴욕페페로니-야탑점",  "홍싸롱-수제돈까스&파스타",  "쇼부덮밥",  "카산도-정자본점",  "부성초밥-미금본점",  "닭장수섭삼계탕&전기구이통닭",  "깨돌이김밥-미금점",  "울진죽변항",  "GTS버거-분당정자점",  "코브라독스-분당미금역점",  "마라하오-죽전점",  "땅끝수산",  "초밥대통령-용인수지점",  "올데이케밥&샐러드",  "야키토리 카마쿠라",  "비오키친-서현본점",  "부산오빠1인국밥-분당점",  "알촌",  "밈피자",  "고씨네카레",  "홍스족발",  "뜸들이다",  "감성커피",  "짚신스시",  "떡형",  "39도시락",  "피제이피자",  "황궁쟁반짜장",  "OK반점",  "모모타코야키&카페",  "꼬알라파이",  "진지덮밥",  "피자프라텔로",  "한식세끼1인김치찜&김치찌개",  "윤희횟집",  "시민보쌈족발&감자탕",  "카페봄봄",  "킹프레소빅와플",  "최가네한쌈",  "창타이누들",  "보돌미역",  "홍대개미"  ]


In [79]:
store_name = []

for store in ygy_store_lst:
    pat = re.split(r'[- &]', store)
    store_name.append(pat[0])

len(store_name)

125

In [81]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813255 sha256=7e92df21c8eb77535ac5b7b4132adfccd14aec2560a58833f4fe9e87d319571f
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model


In [82]:
import kiwipiepy
kiwi = kiwipiepy.Kiwi()

In [83]:
tokens = kiwi.analyze(store_name)

remove_words = []

for token in tokens :
  word = token[0][0][0].form
  pos = token[0][0][0].tag
  if pos in ('NNG') and len(word) ==2 :
    print(word)
    remove_words.append(word)

# 2글자 일반명사 단어들은 제외

춘리
청년
배달
토핑
조형
여왕
모어
육회
갈비
피자
행복
명작
인생
고기
청춘
헬로
대찌
구래
자성
수락
나인
땡초
피자
과일
삼다
최고
걸작
직화
토핑
천년
수제
오늘
고기
공주
홍싸
쇼부
부성
초밥
감성
짚신
진지
한식
시민
카페
보도


In [94]:
korean_particles = ['은', '는', '이', '가', '을', '를', '의', '한', '로', '으로', '까지', '이며', '며', '와', '과']
remove_withparticles = []

for word in remove_words :

  for particle in korean_particles:
    new_word = word + particle
    remove_withparticles.append(new_word)

In [95]:
def replace_store_names(text):
    for name1 in store_name:
        # 먼저 가게 이름 전체가 텍스트에 포함되어 있는지 확인
        if name1 in text:
            # 가게 이름 전체가 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<store>", text)
        else:
            # 가게 이름의 일부가 텍스트에 포함되어 있는 경우를 확인
            for i in range(len(name1), 1, -1):
                part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
                if part in text:
                    # 가게 이름의 일부가 텍스트에 포함되어 있고, 해당 부분이 remove_words에 없으며,
                    # 해당 부분 뒤에 조사가 붙지 않은 경우에만 대체
                    if part not in remove_words + remove_withparticles :
                        pattern = re.compile(re.escape(part), re.IGNORECASE)
                        text = pattern.sub("<store>", text)
                    break  # 가게 이름 일부를 대체했으면 루프 종료
    return text


In [96]:
train = pd.DataFrame(train)

for i, review in tqdm(enumerate(train['reply']), desc="Processing", ncols=100):
    train['reply'][i] = replace_store_names(review)

Processing: 0it [00:00, ?it/s]<ipython-input-96-18f368f004df>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'][i] = replace_store_names(review)
Processing: 3000it [00:01, 1708.52it/s]


In [91]:
train = pd.DataFrame(train)

# for i, review in tqdm(enumerate(train['review']), desc="Processing", ncols=100):
#     train['review'][i] = replace_store_names(review)


train['review'] = train['review'].apply(replace_store_names)


TypeError: ignored

In [93]:
train['review'].astype(str)

0       ['맛있어요', '기름을', '치즈돈까스가', '너무', '덮밥은', '못먹을정도로...
1                                        ['맛있게', '먹었습니다']
2       ['사장님이', '밥이지만', '고객대하는', '관리하시는거', '생각할수잇는', ...
3                              ['맛있어요', '최고', '정말', '너무']
4       ['맛있어서', '존맛입니다', '탕수육', '먹을게요', '시켰을때', '시켜봤는...
                              ...                        
2995    ['번창하세요', '잘먹었습니다', '맛있게', '왠만하면', '사장님', '리뷰안...
2996    ['전화도', '전화', '콜라는', '없네요', '안받으시고', '어떻게하신건지'...
2997                              ['쫄면은', '최애맛집', '자성당이']
2998              ['먹어도', '맛도', '좋아요', '않고', '매번', '질리지']
2999    ['야채가', '먹었습니다', '맛있게', '아쉬었어요', '다른건', '하나도',...
Name: review, Length: 3000, dtype: object

- 지점명 처리

In [ ]:
# 가게명에서 지점명 추출하기

store_etc_name = []

for store in ygy_store_lst:
    pat = re.split(r'[- &]', store)
    for name in pat[1:]:
        if name != '':
            store_etc_name.append(name)

ygy_region_name = []

for region in store_etc_name:
  if region[-1] =='점':
    ygy_region_name.append(region)

In [ ]:
region_name = ygy_region_name

In [ ]:
def replace_region_names(text):
    for name1 in region_name:
        if name1 in text:
            # 지점 이름이 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<region>", text)
        # else :
        #   for i in range(len(name1), 1, -1):
        #       part = name1[:i]  # 지점 이름에서 처음부터 i 글자까지의 부분
        #       if part in text:
        #           # 지점 이름의 일부가 텍스트에 포함되어 있다면 대체
        #           pattern = re.compile(re.escape(part), re.IGNORECASE)
        #           text = pattern.sub("<region>", text)
    return text


In [ ]:
for i, review in tqdm(enumerate(train['reply']), desc="Processing", ncols=100):
    train['reply'][i] = replace_region_names(review)

In [ ]:
for i, review in tqdm(enumerate(train['review']), desc="Processing", ncols=100):
    train['review'][i] = replace_region_names(review)

- 메뉴명 처리

In [ ]:
ygy_menu_list = ['도시락','카레','커피','국밥','마라탕','마라샹궈','우동','나가사키','연어','치킨','쌀국수','닭강정','김치찜','김치찌개',
                '족발','보쌈','육회','갈비','피자','찜닭','파스타','냉면','곱창','닭발','막창','반찬','곱도리탕','순대','떡볶이','부대찌개'
                '삼겹살','분식','라면','버거','덮밥','초밥','통닭','짜장','짬뽕','탕수육','카페','와플','누들','김치','온면','디저트','라떼','고기','덮밥',
             '새우','돼지','닭','찌개','밥',
                 '양파','숙주','고수'] # 재료명 제거

In [ ]:
def replace_menu_names(text):
    # 메뉴 이름을 긴 것부터 짧은 것 순으로 정렬
    sorted_menu_names = sorted(menu_name, key=len, reverse=True)

    for name1 in sorted_menu_names:
        if name1 in text:
            # 메뉴 이름이 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<menu>", text)
    return text


In [ ]:
for i, review in tqdm(enumerate(train['reply']), desc="Processing", ncols=100):
    train['reply'][i] = replace_menu_names(review)

# 사전학습 모델

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "skt/kogpt2-base-v2"
# model_name = "EasthShin/Youth_Chatbot_Kogpt2-base"

model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

tokenizer.eos_token_id # eos 토큰의 idx 반환

In [ ]:
tokenizer.tokenize(train['review'][0])